In [1]:
from tensorflow.keras import layers, models, Model
import numpy as np
import pandas as pd

In [2]:
# Import the data
path = 'https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_3/datasets/wine_quality.csv'
df = pd.read_csv(path)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.2,0.39,0.63,11.0,0.044,55.0,156.0,0.99740,3.09,0.44,8.7,ok,white
1,6.9,0.63,0.02,1.9,0.078,18.0,30.0,0.99712,3.40,0.75,9.8,ok,red
2,6.9,0.30,0.33,4.1,0.035,26.0,155.0,0.99250,3.25,0.79,12.3,good,white
3,7.3,0.42,0.38,6.8,0.045,29.0,122.0,0.99250,3.19,0.37,12.6,good,white
4,6.9,0.18,0.38,8.1,0.049,44.0,176.0,0.99580,3.30,0.54,9.8,ok,white


We have two choices; should the y columns of quality and color all be predicted in a single layer? It might be easier to separate these into two layers, both having the sigmoid activation function. To start, lets preprocess these layers using labelencoder and onehotencoder.

In [4]:
# Preprocess y
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Preprocess "quality" column (one-hot encoding)


# Preprocess "color" column (label encoding for binary; one-hot encoding for multiple categories)


# Concatenate the encoded columns to the original DataFrame


# Drop the original "quality" and "color" columns



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,color_encoded,quality_bad,quality_good,quality_ok
0,7.2,0.39,0.63,11.0,0.044,55.0,156.0,0.99740,3.09,0.44,8.7,1,0.0,0.0,1.0
1,6.9,0.63,0.02,1.9,0.078,18.0,30.0,0.99712,3.40,0.75,9.8,0,0.0,0.0,1.0
2,6.9,0.30,0.33,4.1,0.035,26.0,155.0,0.99250,3.25,0.79,12.3,1,0.0,1.0,0.0
3,7.3,0.42,0.38,6.8,0.045,29.0,122.0,0.99250,3.19,0.37,12.6,1,0.0,1.0,0.0
4,6.9,0.18,0.38,8.1,0.049,44.0,176.0,0.99580,3.30,0.54,9.8,1,0.0,0.0,1.0


In [5]:
# Split data into X and two separate y variables



# Split data into training and testing sets
from sklearn.model_selection import train_test_split



Now that the data is processed, start by creating the shared portion of the model. We start with the input layer, but note that because we are note using the "sequential" model from keras, we must specify where each new layer attaches to the model. We do this by placing the name of the preceding layer in parentheses at the end of the line creating the new layer.

Note how shared_layer1 is created as a Dense layer, and then (input_layer) specifies that shared_layer1 will come directly after the input_layer. In the following line, shared_layer2 is created to follow shared_layer1.

In [6]:
# Create the shared layers of the model

# Input layer

# Shared hidden layers


So far, we've created a sequential set on layers, one following the other. Now we will create two branches for our two output layers. To do this, we create two layers and specify the same "preceding layer" for each. Note how both the quality_output layer and the color_output layer connect to the model via shared_layer2.

When creating these layers, we determine that sigmoid is best for quality and for color. That said, there are arguments to be made for either!

In [7]:
# Branch for quality prediction

# Branch for color prediction


Now we can pull the model together. We only need to specify the input and output layers and Keras will infer the rest. To compile the model, we can specify unique metrics and loss functions for each output layer, but for this dataset we have chosen 'binary_crossentropy' as the loss function and 'accuracy' as the metric for both output layers.

In [8]:
# Create the model

# Compile the model


# Display the model summary


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        768 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ quality_output      │ (None, 3)         │         99 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_output        │ (None, 1)         │         33 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,980 (11.64 KB)

 Trainable params: 2,980 (11.64 KB)

 Non-trainable params: 0 (0.00 B)

To fit the model to the data, we specify X_train as normal, but pass a dictionary for the y_data.

In [9]:
# Fit the model


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - color_output_accuracy: 0.7718 - loss: 3.9928 - quality_output_accuracy: 0.7266 - val_color_output_accuracy: 0.9246 - val_loss: 0.9401 - val_quality_output_accuracy: 0.7154
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - color_output_accuracy: 0.9266 - loss: 0.8277 - quality_output_accuracy: 0.7550 - val_color_output_accuracy: 0.9254 - val_loss: 0.9371 - val_quality_output_accuracy: 0.7446
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - color_output_accuracy: 0.9424 - loss: 0.7833 - quality_output_accuracy: 0.7666 - val_color_output_accuracy: 0.9385 - val_loss: 0.8356 - val_quality_output_accuracy: 0.7462
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - color_output_accuracy: 0.9483 - loss: 0.7747 - quality_output_accuracy: 0.7584 - val_color_output_accuracy: 0.9431 - val_loss: 0.7896 - val_quality_output_accuracy: 0.7469
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - color_output_accuracy: 0.9578 - loss: 0.73

In [10]:
# Evaluate the model with the testing data


51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - color_output_accuracy: 0.9456 - loss: 0.7447 - quality_output_accuracy: 0.7626


[0.7330668568611145, 0.9520000219345093, 0.7729230523109436]